In [3]:
import heapq  # Importing the heapq module for priority queue operations
import numpy as np  # Importing NumPy for array manipulations

def get_user_input():
    """ Get user input for the puzzle configuration """
    print("Enter the puzzle configuration as a 3x3 grid (use 0 for empty space):")
    board = []  # Initialize an empty list to store the puzzle
    for i in range(3):  # Loop through 3 rows
        row = list(map(int, input().split()))  # Read input row and convert to integers
        board.append(row)  # Append the row to the board
    return np.array(board)  # Convert list to NumPy array and return

class Puzzle:
    """ Represents a state of the 8-puzzle """
    def __init__(self, board, parent=None, move="", depth=0, cost=0):
        self.board = board  # The current puzzle configuration
        self.parent = parent  # Parent node to track the solution path
        self.move = move  # Move taken to reach this state
        self.depth = depth  # Depth in the search tree
        self.cost = cost  # Estimated cost based on heuristic

    def __lt__(self, other):
        """ Compare nodes based on total cost (A* priority queue) """
        return (self.depth + self.cost) < (other.depth + other.cost)

    def __eq__(self, other):
        """ Check if two puzzle states are the same """
        return np.array_equal(self.board, other.board)

def heuristic(board, goal):
    """ Manhattan Distance heuristic function """
    distance = 0  # Initialize distance counter
    for i in range(3):  # Iterate over rows
        for j in range(3):  # Iterate over columns
            if board[i][j] != 0:  # Ignore empty space (0)
                x, y = np.where(goal == board[i][j])  # Find target position in the goal state
                distance += abs(i - x[0]) + abs(j - y[0])  # Compute Manhattan distance
    return distance  # Return total heuristic cost

def get_neighbors(node, goal):
    """ Generate possible moves from the current state """
    moves = []  # List to store possible moves
    x, y = np.where(node.board == 0)  # Locate empty space (0)
    x, y = int(x[0]), int(y[0])  # Convert coordinates to integers

    # Define possible moves with corresponding row and column shifts
    directions = {"Up": (-1, 0), "Down": (1, 0), "Left": (0, -1), "Right": (0, 1)}

    for move, (dx, dy) in directions.items():  # Iterate through possible moves
        new_x, new_y = x + dx, y + dy  # Calculate new position

        # Check if the move is within bounds
        if 0 <= new_x < 3 and 0 <= new_y < 3:
            new_board = node.board.copy()  # Copy the board to avoid modifying the original
            new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]  # Swap tiles
            moves.append(Puzzle(new_board, node, move, node.depth + 1, heuristic(new_board, goal)))  # Create new node

    return moves  # Return list of possible moves

def solve_puzzle(start, goal):
    """ Solve the puzzle using A* search algorithm """
    start_node = Puzzle(start, None, "", 0, heuristic(start, goal))  # Initialize start node
    open_list = []  # Priority queue for nodes to explore
    closed_set = set()  # Set to track visited states

    heapq.heappush(open_list, start_node)  # Add start node to priority queue

    while open_list:  # Continue until there are nodes to explore
        current_node = heapq.heappop(open_list)  # Get the node with the lowest cost

        if np.array_equal(current_node.board, goal):  # Check if goal state is reached
            path = []  # List to store solution path
            while current_node.parent:  # Trace back moves from goal to start
                path.append(current_node.move)  # Add move to path
                current_node = current_node.parent  # Move to parent node
            return path[::-1]  # Return moves in the correct order

        closed_set.add(str(current_node.board))  # Mark node as visited

        for neighbor in get_neighbors(current_node, goal):  # Iterate through neighbors
            if str(neighbor.board) not in closed_set:  # Only consider unvisited states
                heapq.heappush(open_list, neighbor)  # Add new state to priority queue

    return None  # No solution found

if __name__ == "__main__":
    print("Enter the start state:")
    start_state = get_user_input()  # Read user input for start state
    print("Enter the goal state:")
    goal_state = get_user_input()  # Read user input for goal state

    solution = solve_puzzle(start_state, goal_state)  # Solve the puzzle
    if solution:
        print("Solution found in", len(solution), "moves:", solution)  # Print solution steps
    else:
        print("No solution possible.")  # Print failure message if no solution exists

Enter the start state:
Enter the puzzle configuration as a 3x3 grid (use 0 for empty space):
1 2 3
4 0 5
6 7 8
Enter the goal state:
Enter the puzzle configuration as a 3x3 grid (use 0 for empty space):
1 2 3
4 5 6
7 8 0
Solution found in 14 moves: ['Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right']
